In [ ]:
# 2024/11/19
# playing around with trying to see a spatial relationship with when points start or stop moving
# I'm not really seeing that, so this can likely be deleted

In [ ]:
from hsflfm.util import load_dictionary
from hsflfm.analysis import ResultManager, ResultPlotter, MetadataManager

from matplotlib import pyplot as plt
import numpy as np

In [ ]:
MetadataManager.all_specimen_numbers()

In [ ]:
specimen_name = "20240506_OB_3"
strike_number = 8

In [ ]:
result_info = load_dictionary(f"../temporary_result_storage_3/{specimen_name}/strike_{strike_number}_results.json")

In [ ]:
rm = ResultManager(result_info)
plotter = ResultPlotter(result_info)

In [ ]:
values = rm.peak_indices()
_ = plotter.scatter_values(values, cmap='turbo')
plotter.scatter_over_image(values)
_ = plotter.scatter_peak_disp()
plotter.plot_all_displacement()

In [ ]:
# figure out when points reach 10% of max value 
rm.peak_displacements()

In [ ]:
z_disp = rm.rel_displacements[:, :, 2]
strike_index = int(rm.strike_center_index())
point_cross_indices = []
for i, p in enumerate(z_disp):
    plt.plot(p, color='black')
    maxv = rm.peak_displacements()[i]
    thresh = maxv / 10
    #if abs(thresh) < 0.003:
    #    point_cross_indices.append(np.nan)
    #    continue
    #plt.axhline(y=thresh)

    diff = p - thresh 
    crossing_indices = np.where(np.diff(np.sign(diff)))[0]
    closest_cross = crossing_indices[np.argmin(np.abs(crossing_indices - strike_index))]

    #print(crossing_indices, closest_cross)
    #for i in crossing_indices:
    p1, p2 = p[closest_cross], p[closest_cross + 1] 
    crossing = closest_cross + (thresh - p1) / (p2 - p1)
    plt.axvline(x=crossing)
    point_cross_indices.append(float(crossing))
    #break

plt.axvline(x=rm.strike_center_index())

In [ ]:
start_locs = rm.point_start_locs_ant_mm
plt.scatter(start_locs[:, 1], start_locs[:, 0], c=point_cross_indices,
            clim=(16, 18)
            )
plt.colorbar()

In [ ]:
start_locs.shape